In [2]:
import random
import keyring

from neo4j import GraphDatabase
from neomodel.contrib.spatial_properties import NeomodelPoint
from neomodel import config, db

from faker import Faker

from utils.user_creation import Create_Person
from utils.neomodel_models import Geo, Person

In [3]:
# Set passwords 
#keyring.set_password('neo4j', 'login', 'neo4j')
#keyring.set_password('neo4j', 'password', '')
#keyring.set_password('neo4j', 'uri', 'bolt://localhost:7687')

In [4]:
# Pull login information
login = keyring.get_password('neo4j', 'login')
password = keyring.get_password('neo4j', 'password')
uri = keyring.get_password('neo4j', 'uri')

In [5]:
# Create driver 
my_driver = GraphDatabase().driver(uri, auth=(login, password))
config.DRIVER = my_driver

# Generate fake users & locations

In [6]:
fake = Faker()

In [7]:
# Generate people 
with db.transaction: 
    for i in range(0, 1000): 
        p1 = Create_Person()
        neo4j_person = Person(**p1.cypfer_query())
        neo4j_person.save()

In [8]:
# Generate geo data 
for i in range(0, 1000):
    with db.transaction:
        lat = float(round(fake.latitude(), 2))
        long = float(round(fake.latitude(), 2))
        loc = Geo(location=NeomodelPoint((lat, long), crs='wgs-84'))
        loc.save()

# Connect users to geo

In [9]:
# Create connections 
people = Person.nodes.all()
locations = Geo.nodes.all()

with db.transaction:
    for person in people: 
        num_geo = random.randint(1, 10)
        random_sample_geo = random.sample(locations, num_geo)
        for geo in random_sample_geo:
            person.geo_data.connect(geo)
        

# Query where geo locations associated with more than one user

In [10]:
qry_more_than_one_user = '''MATCH (geo:Geo) 
WHERE COUNT {(geo)-[:LOCATED_AT]-(:Person)} > 8
RETURN geo'''

In [11]:
results, meta = db.cypher_query(qry_more_than_one_user)
results

[[<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1001' labels=frozenset({'Geo'}) properties={'location': POINT(-38.63 -26.02)}>],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1035' labels=frozenset({'Geo'}) properties={'location': POINT(-41.7 -52.48)}>],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1053' labels=frozenset({'Geo'}) properties={'location': POINT(-47.99 13.08)}>],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1057' labels=frozenset({'Geo'}) properties={'location': POINT(4.12 -29.39)}>],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1071' labels=frozenset({'Geo'}) properties={'location': POINT(-19.82 -39.25)}>],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1088' labels=frozenset({'Geo'}) properties={'location': POINT(-41.73 79.93)}>],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1090' labels=frozenset({'Geo'}) properties={'location': POINT(69.86 0.19)}>],
 [<Node element_id='4:987c64

In [12]:
qry_nodes_near_location = """MATCH (geo:Geo)
WHERE point.distance(
 geo.location,
 point({latitude:4.83, longitude:64.05})) < 2000000
RETURN geo, point.distance(
 geo.location,
 point({latitude:4.83, longitude:64.05}))"""

In [13]:
results2, meta  = db.cypher_query(qry_nodes_near_location)
results2

[[<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1036' labels=frozenset({'Geo'}) properties={'location': POINT(69.97 -2.55)}>,
  1052828.676578961],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1039' labels=frozenset({'Geo'}) properties={'location': POINT(67.28 18.78)}>,
  1592074.5714118853],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1058' labels=frozenset({'Geo'}) properties={'location': POINT(72.34 -6.3)}>,
  1543987.5349072688],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1079' labels=frozenset({'Geo'}) properties={'location': POINT(52.01 7.62)}>,
  1367943.318022243],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1085' labels=frozenset({'Geo'}) properties={'location': POINT(48.87 1.0)}>,
  1740346.762794677],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079d0198decc:1090' labels=frozenset({'Geo'}) properties={'location': POINT(69.86 0.19)}>,
  827085.3231758048],
 [<Node element_id='4:987c64da-a00c-4ad0-ac12-079